In [1]:
# !export OPENAI_API_KEY = key
# !pip install openai
import pandas as pd
from openai import OpenAI
import os

In [ ]:


key = "sk-proj-"

client = OpenAI(api_key = key)

def extract_entity(sentence):
    completion = client.chat.completions.create(
    model="gpt-4o",
    temperature= 0,
    messages = [
    {"role": "system", "content": """You are a virtual annotation. For each sentence, you annotate the addresser, addressee, actionResult, date. 
    The output shoulbe be in Json format.
    Adresser: Who performs the action.
    Action: What action is performed.
    ActionResult: What is done.
    Addressee: To whom the action is directed.
    Date: When the action occurs/deadline."""},
    
    {"role": "user", "content": """By 31 December 2010 and, thereafter, at least every three years, the Commission shall review the provisions concerning its implementing powers and present a report to the European Parliament and to the Council on the functioning of those powers. The report shall examine, in particular, the need for the Commission to propose amendments to this Directive in order to ensure the appropriate scope of the implementing powers conferred on the Commission. The conclusion as to whether or not an amendment is necessary shall be accompanied by a detailed statement of reasons. If necessary, the report shall be accompanied by a legislative proposal to amend the provisions conferring implementing powers on the Commission."""},
    {"role": "system", "content": """{
    "addresser": ["the commission"],
    "Action": ["present"],
    "ActionResult": ["a report on the functioning of those powers"],
    "addressee": ["the european parliament", "the council"],
    "Date": ["By 31 December 2010"]
    }"""},
    
     {"role": "user", "content": """Member States shall communicate to the Commission the texts of the main provisions of national law which they adopt in the field governed by this Directive."""},
    {"role": "system", "content": """{
    "addresser": ["member states"],
    "Action": ["communicate"],
    "ActionResult": ["the texts of the main provisions of national law"],
    "addressee": ["the Commission"],
    "Date": ["None"]
    }"""},
    
     {"role": "user", "content": """Upon reasoned request, Member States shall forthwith communicate the reports referred to in Article 111(3) to the competent authorities of another Member State."""},
    {"role": "system", "content": """{
    "addresser": ["Member States"],
    "Action": ["communicate"],
    "ActionResult": ["the reports referred to in Article 111(3)"],
    "addressee": ["the competent authorities of another Member State"],
    "Date": ["forthwith"]
    }"""},
    
    {"role": "user", "content": """Each year the sponsor shall submit to the Agency a report on the state of development of the designated medicinal product."""},
    {"role": "system", "content": """{
    "addresser": ["the sponsor"],
    "Action": ["submit"],
    "ActionResult": ["a report on the state of development of the designated medicinal product"],
    "addressee": ["the Agency"],
    "Date": ["Each year"]
    }"""},
    
    {"role": "user", "content": sentence},
]
)
    return completion.choices[0].message.content

def saveToExcelFile(sentences, responses, save_dir, writing_mode = "w+"):
    df = pd.DataFrame({'Sentence': [sentences], 'Output': [responses]})
        # Check if the file exists
    filename = "../data/llm_output/" + save_dir + ".xlsx"
    if not os.path.exists(filename):
        # print('file not existing')
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Sheet1', index=False)
    else:
        # print('file exist')
        # Append to the existing file
        with pd.ExcelWriter(filename, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, sheet_name='Sheet1', index=False, header=False, startrow=writer.sheets['Sheet1'].max_row if 'Sheet1' in writer.sheets else 0)
            
def prompting(sentence, output_filename):
    responses = extract_entity(sentence)
    saveToExcelFile(sentence, responses, output_filename)
    
# print(completion.choices[0].message)

In [ ]:
df = pd.read_excel("../data/manually_selection.xlsx", sheet_name = "Sheet1")
output_log_filename = "openai_entities_extraction"
df = df[189:-1]
print(df.iloc[0]['Sentence'])
for _, paragraph in df.iterrows():
    requirement = prompting(paragraph['Sentence'], output_log_filename)
print("### Finish ###")

Member States shall communicate to the Commission the text of the main provisions of national law which they adopt in the field covered by this Directive.
